In [ ]:
!pip install -q jupyter-dash==0.3.0rc1 dash-bootstrap-components transformers
!pip install transformers[sentencepiece]
!pip install dash-daq
!pip install nltk

     |████████████████████████████████| 45 kB 1.2 MB/s 
     |████████████████████████████████| 209 kB 9.6 MB/s 
     |████████████████████████████████| 3.4 MB 46.6 MB/s 
     |████████████████████████████████| 7.3 MB 35.4 MB/s 
     |████████████████████████████████| 26.5 MB 1.2 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 357 kB 59.4 MB/s 
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 642 kB 5.4 MB/s 
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669714 sha256=5a5c464a99f59d8e66b48a650ff2900ee9a8b6c5db16ede5ae1981b52dc799ef
  Stored in directory: /root/.cache/pip/wheels/fe/54/53/a8d448db5592874db4313240571ca2c069e55f6a6b29bf5847
Successfully built dash-daq


In [ ]:
!pip install "dash-bootstrap-components<1"

     |████████████████████████████████| 197 kB 5.3 MB/s 
  Attempting uninstall: dash-bootstrap-components
    Found existing installation: dash-bootstrap-components 1.0.2
    Uninstalling dash-bootstrap-components-1.0.2:
      Successfully uninstalled dash-bootstrap-components-1.0.2


In [ ]:
import time
import dash_table
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import dash_daq as daq
from jupyter_dash import JupyterDash
from transformers import BartTokenizer, BartForConditionalGeneration , pipeline
import torch
import re
from bs4 import BeautifulSoup
import requests
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx 
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Load Model
bart_tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
bart_model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
#bart_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Colab Notebooks/NewsFeedSummary/bart_cnn126", local_files_only=True)


t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
#t5_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Colab Notebooks/NewsFeedSummary/t5_base", local_files_only=True)



Device: cpu


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
#model_save_name = 'bart_cnn126'
#path = F"/content/drive/MyDrive/Colab Notebooks/NewsFeedSummary/{model_save_name}"
#bart_model.save_pretrained(path)

#model_save_name = 't5_base'
#path = F"/content/drive/MyDrive/Colab Notebooks/NewsFeedSummary/{model_save_name}"
#t5_model.save_pretrained(path)

# New Section

In [ ]:
# Define app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Output Length (# Tokens)"),
                dcc.Slider(
                    id="max-length",
                    min=20,
                    max=160,
                    value=60,
                    marks={i: str(i) for i in range(20, 161, 20)},
                ),
            ]
        ),dbc.FormGroup(
            [
                dbc.Label("Beam Size"),
                dcc.Slider(
                    id="num-beams",
                    min=2,
                    max=6,
                    value=4,
                    marks={i: str(i) for i in [2, 4, 6]},
                ),
            ]
        ),
     
        dbc.FormGroup(
            [
                dbc.Spinner(
                    [   dbc.Row([dbc.Col(
                                  width=5,
                                  children=[dbc.Button("Summarize", id="button-run")]),
                        daq.BooleanSwitch(id='chunk-switch', on=False,label="Enable Chunks",labelPosition="top")])
                    ,
                        html.Div(id="time-taken"),
                    ]
                )
            ]
        ),
    ],
    body=True,
    style={"height": "300px"},
)



table=html.Div([
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {'id': 'model', 'name': 'Model Name'},
            {'id': 'time', 'name': 'Time Taken'},
        ],
        data=[  {"model": "Distilbart-cnn-12-6", "time": '-'},
                {"model": "T5_Base", "time": '-'},
                {"model": "Extractive", "time": '-'},
              ],
        row_selectable="single",
        selected_rows=[0],
         editable=True,
    ),
    html.Div(id='datatable-interactivity-container')
])


# Define Layout
app.layout = dbc.Container(
    fluid=True,
    children=[
        html.H1("News Feed Automatic Summarization"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(
                    width=5,
                    children=[
                        controls,
                        dbc.Card(
                            body=True,
                            children=[
                                dbc.FormGroup(
                                    [
                                        dbc.Label("Summarized Content"),
                                        dcc.Textarea(
                                            id="summarized-content",
                                            style={
                                                "width": "100%",
                                                "height": "calc(75vh - 300px)",
                                            },
                                        ),
                                    ]
                                )
                            ],
                        ),
                    ],
                ),
                dbc.Col(
                    width=7,
                    children=[
                        dbc.Card(
                            body=True,
                            children=[
                                table,
                                dbc.Card(
                                body=True,
                                children=[
                                dbc.FormGroup(
                                    [
                                        dbc.Label("Original Text (Paste here)"),
                                        dcc.Textarea(
                                            id="original-text",
                                            style={"width": "100%", "height": "calc(75vh - 125px)"},
                                        ),
                                    ]
                                )
                              ],
                            )
                          ],
                        )
                    ],
                ),
            ]
        ),
    ],
)

In [ ]:
@app.callback(
    [Output("summarized-content", "value"), Output("time-taken", "children"),Output('datatable-interactivity', 'data')],
    [
        Input("button-run", "n_clicks"),
        Input("max-length", "value"),
        Input("num-beams", "value"),
        Input('datatable-interactivity', "selected_rows"),
        Input('datatable-interactivity', 'data_timestamp'),
        Input('chunk-switch', 'on')
        
    ],
    [State("original-text", "value"),State('datatable-interactivity', 'data')],
)
def summarize(n_clicks, max_len,num_beams,selected_rows,timestamp,chunk_switch ,original_text,rows):
    if original_text is None or original_text == "":
        return "", "Did not run"

    #extractive code     
    stop_words = stopwords.words('english')
    def text_rank(text):
        sentences = nltk.sent_tokenize(text) # this gives us a list of sentences
        # remove punctuations, numbers and special characters
        clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

        # make alphabets lowercase
        clean_sentences = [s.lower() for s in clean_sentences]
        def remove_stopwords(sen):
            sen_new = " ".join([i for i in sen if i not in stop_words])
            return sen_new
        clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
        w2v=Word2Vec(clean_sentences,size=1,min_count=1,iter=1000)
        sentence_embeddings=[[w2v[word][0] for word in words] for words in clean_sentences]
        max_len=max([len(tokens) for tokens in clean_sentences])
        sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
        similarity_matrix = np.zeros([len(clean_sentences), len(clean_sentences)])
        for i,row_embedding in enumerate(sentence_embeddings):
            for j,column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph)
        top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
        top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
        final=""
        for sent in sentences:
            if sent in top.keys():
                final=final+sent
        return final 
    #Screapping
    ree=re.findall(r'^https',original_text)
    if ree :
        URL = original_text
        r = requests.get(URL)
        soup1=BeautifulSoup(r.text,'html.parser')
        result1= soup1.find_all("div",{"data-component": "text-block"})
        text = [results.text for results in result1]
        article = ' '.join(text)
        original_text = article

    word_list = original_text.split()
    o_number_of_words = len(word_list)

    if(chunk_switch==True):
        t0 = time.time()
        #1st model
        if selected_rows[0]==0:
            sentences = nltk.tokenize.sent_tokenize(original_text)
            # initialize
            length = 0
            chunk = ""
            chunks = []
            count = -1
            for sentence in sentences:
                count += 1
                combined_length = len(bart_tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

                if combined_length  <= bart_tokenizer.max_len_single_sentence: # if it doesn't exceed
                    chunk += sentence + " " # add the sentence to the chunk
                    length = combined_length # update the length counter

                    # if it is the last sentence
                    if count == len(sentences) - 1:
                        chunks.append(chunk.strip()) # save the chunk
                  
                else: 
                    chunks.append(chunk.strip()) # save the chunk
                    
                    # reset 
                    length = 0 
                    chunk = ""

                    # take care of the overflow sentence
                    chunk += sentence + " "
                    length = len(bart_tokenizer.tokenize(sentence))
            
            # inputs to the model
            inputs = [bart_tokenizer(chunk, return_tensors="pt",add_special_tokens=True) for chunk in chunks]
            out=""
            for input in inputs:
                output = bart_model.generate(**input,num_beams=num_beams, min_length=max_len, length_penalty=2.0, early_stopping=True)
                out +=bart_tokenizer.decode(*output, skip_special_tokens=True)
      
            model_name='Distilbart-cnn-12-6'

        #second model
        elif selected_rows[0]==1:
            sentences = nltk.tokenize.sent_tokenize(original_text)
            # initialize
            length = 0
            chunk = ""
            chunks = []
            count = -1
            for sentence in sentences:
                count += 1
                combined_length = len(t5_tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

                if combined_length  <= t5_tokenizer.max_len_single_sentence: # if it doesn't exceed
                    chunk += sentence + " " # add the sentence to the chunk
                    length = combined_length # update the length counter

                # if it is the last sentence
                if count == len(sentences) - 1:
                    chunks.append(chunk.strip()) # save the chunk
                
                else: 
                    chunks.append(chunk.strip()) # save the chunk
                
                # reset 
                length = 0 
                chunk = ""

                # take care of the overflow sentence
                chunk += sentence + " "
                length = len(t5_tokenizer.tokenize(sentence))
        
            # inputs to the model
            inputs = [t5_tokenizer(chunk, return_tensors="pt") for chunk in chunks]
            out=""
            for input in inputs:
                output = t5_model.generate(**input,num_beams=num_beams, min_length=max_len, max_length=max_len+20 , repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
                out += t5_tokenizer.decode(*output, skip_special_tokens=True)
            
            model_name='T5_Base'
        else :
            out = text_rank(original_text)
            model_name='Extractive'

        t1 = time.time()
     
    else:
        t0 = time.time()
        if selected_rows[0]==0:
            inputs = bart_tokenizer( original_text, return_tensors="pt", max_length=512, truncation=True)
            outputs = bart_model.generate(inputs["input_ids"], min_length=max_len, length_penalty=2.0, num_beams=num_beams, early_stopping=True)
            out=bart_tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
            
            model_name='Distilbart-cnn-12-6'
        elif selected_rows[0]==1:
            input_ids = t5_tokenizer.encode(original_text, return_tensors="pt", add_special_tokens=True)
            generated_ids = t5_model.generate(input_ids=input_ids, num_beams=num_beams, min_length=max_len, max_length=max_len+20, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
            out = t5_tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
            
            model_name='T5_Base'
        else :
            out = text_rank(original_text)
            model_name='Extractive'
        t1 = time.time()
      
      

    if selected_rows[0]==0:
        rows[0]['time'] = f"{t1-t0:.2f}s"
        final_summary=out
    elif selected_rows[0]==1:
        rows[1]['time'] = f"{t1-t0:.2f}s"
        final_summary=out
    else:
        rows[2]['time'] = f"{t1-t0:.2f}s"
        final_summary=out
    
    word_list = final_summary.split()

    out_number_of_words = len(word_list)


    time_taken =( f"Summarized by {model_name} Model in {t1-t0:.2f}s\n"
                f"Origial Text Lenth ={o_number_of_words} words\n"
                f"Summary Text Lenth ={out_number_of_words} words")

    return final_summary, time_taken,rows

Run the cell below to run your Jupyter Dash app. Click on the **temporary** URL to access the app.

In [ ]:
app.run_server(mode='external',debug=False,dev_tools_ui=False,dev_tools_props_check=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2022 11:20:33] "GET /_alive_3214e034-4c77-4266-8577-9f4e4c3165ca HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>